#### Importowanie modułów

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision import models, transforms, datasets

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

c:\Users\rafal\.conda\envs\deeplearning\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Zbiory danych, modele, metody TL, hiperparametry

In [ ]:
DATASETS = [
    'CIFAR10'
]
MODELS = [
    models.alexnet(pretrained=True),
    models.vgg16(pretrained=True),
    models.resnet50(pretrained=True)
]
TL_METHODS = [
    #'no_freezing',
    #'classic_tl',
    'fine_tuning'
]

# Hyperparmeters
EPOCHS = 3
BATCH_SIZE = 100

#### Utworzenie Data setów i data loaderów

In [ ]:
def create_datasets_and_dataloaders(dataset_name: str):
    transform = transforms.Compose([
            transforms.Resize(224),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])

    if dataset == 'CIFAR10':
        train_set = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
        train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)

        test_set = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
        test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

    return train_set, test_set, train_loader, test_loader

#### Przetwarzanie modeli do transfer learningu

In [ ]:
def apply_transfer_learning(model, tl_method):
    if tl_method == 'classic_tl':
        for param in model.parameters():
            param.requires_grad = False
        if hasattr(model, 'classifier'):
            for param in model.classifier.parameters():
                param.requires_grad = True
        elif hasattr(model, 'fc'):
            for param in model.fc.parameters():
                param.requires_grad = True
    elif tl_method == 'fine_tuning' or tl_method == 'no_freezing':
            for param in model.parameters():
                param.requires_grad = True

def customize_model(model, num_classes, tl_method):
    if isinstance(model, models.AlexNet):
        dropout = 0.5
        model.classifier = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    elif isinstance(model, models.VGG):
        dropout = 0.5
        model.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, num_classes),
        )

    elif isinstance(model, models.ResNet):
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, num_classes)

    print("Classifier for model has been changed.")
    apply_transfer_learning(model, tl_method)
    print("Transfer learning method:", tl_method, "applied.")

    return model

#### Pętla ucząco walidacyjna

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", device)

def freeze_percentage(model, percentage):
    # Create a list to store gradients and corresponding parameters
    grad_params = []

    # Collect gradients and parameters
    for param in model.parameters():
        if param.requires_grad and param.grad is not None:
            grad_params.append((param.grad.norm(), param))

    # Sort parameters based on gradients (smaller gradients first)
    grad_params.sort(key=lambda x: x[0])

    # Determine the number of parameters to freeze
    num_params_to_freeze = int(percentage * len(grad_params))

    # Freeze the parameters with the smallest gradients
    for i in range(num_params_to_freeze):
        grad_params[i][1].requires_grad = False

for id_dataset, dataset in enumerate(DATASETS):
    print("Using the", dataset, "dataset.")

    train_set, test_set, train_loader, test_loader = create_datasets_and_dataloaders(dataset)

    num_classes = len(train_set.classes)

    for id_tl_method, tl_method in enumerate(TL_METHODS):
        print("Using the", tl_method, 'method')

        for id_model, model in enumerate(MODELS):
            print("Using the", model.__class__.__name__, "model")

            # Model customization (to match the output size)
            model = customize_model(model, num_classes, tl_method)

            # Loss Function and Optimizer
            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=0.001)



            model.to(device)

            # Training and Validation
            print("Training starts")
            for epoch in range(EPOCHS):
                model.train()
                running_loss = 0.0
                num_batches = len(train_set.data) // BATCH_SIZE
                for batch, (images, labels) in enumerate(train_loader):
                    images = images.to(device)
                    labels = labels.to(device)
                    print("\rEpoch:", epoch, "Batch:", batch+1, '/', num_batches, end =' ')
                    optimizer.zero_grad()
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    running_loss += loss.item()

                # Validation
                model.eval()
                val_loss = 0.0
                correct = 0
                total = 0
                with torch.no_grad():
                    for images, labels in test_loader:
                        images, labels = images.to(device), labels.to(device)
                        outputs = model(images)
                        loss = criterion(outputs, labels)
                        val_loss += loss.item()
                        _, predicted = torch.max(outputs.data, 1)
                        total += labels.size(0)
                        correct += (predicted == labels).sum().item()

                print(f'Epoch {epoch+1}, Train Loss: {running_loss/len(train_loader)}, '
                    f'Validation Loss: {val_loss/len(test_loader)}, '
                    f'Accuracy: {100 * correct / total}%')

                # Fine-tuning based on tl_method
                if tl_method == 'fine_tuning':
                    freeze_percentage(model, percentage=0.2)  # Adjust the percentage as needed
                    # Unfreeze all layers for fine-tuning
                    for param in model.parameters():
                        param.requires_grad = True
                    # Adjust learning rate for fine-tuning
                    for param_group in optimizer.param_groups:
                        param_group['lr'] = 0.0001